# 배치할당  

**Info**  

슈트별 작업량(=품번*수량)이 동일하게 물품을 할당하는 알고리즘  
- 모든 슈트가 해방되어야 다음 작업이 수행되도록 구현
- 해방된 슈트는 먼저 다음 작업을 수행할 수 있도록 구현  
-> 2안은 작업시간을 고려하지 못하고,(실제로 동일한 작업량일지라도 작업 시간이 다를 수 있음) 또한 물품이 어떻게 슈트에 할당되는지 알 수 없기 때문에(알고리즘은 주문번호 그룹화로 진행) 실질적으로 사용에 한계가 있음.  
-> 1안은 작업량이 많은 순으로 슈트에 배분되기 때문에 앞선 작업이 더 많은 작업량이 할당됨. 즉, 슈트별 작업량은 균등하더라도 배치별 작업량은 균등하지 않음.  
  
**함수 설명:**  

*1안: 모든 슈트가 해방되어야 다음 배치 수행*
- release_and_process_orders(df, order, count, shut):
    - df: 불러온 원본 데이터
    - order: 주문번호
    - count: 수량
    - shut: 슈트 개수   
    
*2안: 슈트별 독립적으로 작업 할당*  
- process_order_with_released_suit(ddf, order, count, shut):
    - df: 불러온 원본 데이터
    - order: 주문번호
    - count: 수량
    - shut: 슈트 개수

In [ ]:
# 변수 설정

# 칼럼명 고정, 출하일이 없어서 출고일로 대신 지정함.(바꿔서 사용 바람.)
order = '온라인 주문번호'
count = '수량'
sku = '품번'
date = '출고일'
# 슈트 개수
shut = 60

In [ ]:
# 필요한 라이브러리와 데이터 불러오기
# 경로를 알맞게 입력해줘야 함.
import pandas as pd
import time
df = pd.read_excel('data/BA_Sample_Samsung_0421_raw_data.xlsx',usecols= [order,count,sku,date])

In [ ]:
df

#### 알고리즘1: 모든 슈트 해방 후 일괄로 주문 처리 방식


1. 주문번호끼리 그룹화

2. 작업량 = 품번의 개수 * 수량 -> 그룹(주문번호)마다 작업량 계산

3. 작업량이 많은 순으로 정렬(내림차순)(=투입순서)

4. 각 슈트에는 아래와 같이 할당됨(숫자가 작을수록 우선순위, 작업량 많은 항목)(ㄹ모양)  
    <0> <1>          <58> <59>  
0-   0   1    ....    58   59   ->   1번 배치 (0\*60+i+1)   ->   y-0\*60  
1-  119 118   ....    61   60   ->   2번 배치 (2\*60-i)     ->   2\*60-1-y  
2-  120 121   ....   178  179   ->   3번 배치 (2\*60+i+1)   ->   y-2\*60

#### 생성결과

1. 투입 순서 + 배치 순서 

2. 배당되는 슈트

In [ ]:
# 함수 정의 - 수정 불가
def release_and_process_orders(df, order, count, shut,date):
  start_time = time.time()
  for d_value, group in df.groupby(date):
    total_count = 0
    total_sku = 0
    total_batch = 0
    order_df = group.groupby(order)[count].sum().reset_index()
    order_df = order_df.sort_values(by=count, ascending=False).reset_index(drop=True)
    order_df['투입순서'] = order_df.index
    order_df['배치'] = order_df.index // shut
  
    for i in range(len(order_df)):
      if (order_df.loc[i, '배치'] % 2 == 1):
          order_df.loc[i, '슈트'] = shut*(order_df.loc[i, '배치']+1)-i-1
      else:
          order_df.loc[i, '슈트'] = i- order_df.loc[i, '배치']*shut
    order_df.drop(columns=[count], inplace=True)
    merged_group = group.merge(order_df, on=order, how='left')
    merged_group.to_csv(f'orders_{d_value}.csv', index=False)

    for batch_value, batch_group in merged_group.groupby('배치'):
      #print(f"{d_value} - {batch_group['배치']}번째 배치: 작업량 {batch_group[count].sum()}, 품목수 {batch_group[sku].nunique()}")
      total_count += batch_group[count].sum()
      total_sku += batch_group[sku].nunique()
      total_batch += 1
    
    print(f"{d_value}: 총 작업량 {total_count}, 총 품목 수 {total_sku}, 총 배치 수 {total_batch}, 배치별 평균 작업량: {total_count / total_batch}")
    
  end_time = time.time()
  elapsed_time = end_time - start_time
  print(f"총 소요 시간: {elapsed_time:.2f}초")
  print("모든 주문 해방 후 일관되게 주문을 처리하는 방식으로 날짜별 파일에 저장되었습니다.")

In [ ]:
# 함수 실행
release_and_process_orders(df, order, count, shut,date)

#### 알고리즘2: 하나의 슈트가 해방될 경우 해당 슈트는 다음 주문 처리


1. 주문번호끼리 그룹화

2. 작업량 = 품번의 개수 * 수량 -> 그룹(주문번호)마다 작업량 계산

3. 총작업량 계산 후 각 슈트당 작업량 할당

4. 작업을 남아있는 작업량이 가장 많은 슈트에 할당, 남아있는 작업량 업데이트

In [ ]:
# 함수 정의 - 수정 불가
def process_order_with_released_suit(df, order, count, shut,date):
    start_time = time.time()
    for d_value,group in  df.groupby(date):
      order_df = df.groupby(order)[count].sum().reset_index()
      current_workload = [order_df['수량'].sum()/shut]*shut
      for i in range(len(order_df)):
          index = current_workload.index(max(current_workload))
          order_df.loc[i, '슈트'] = index
          current_workload[index] -= order_df.loc[i, count]
      order_df.to_csv(f'ba_date{d_value}2.csv', index=False)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"총 소요 시간: {elapsed_time:.2f}초")
    print("하나의 슈트가 해방될 경우 해당 슈트는 다음 주문 처리하는 방식으로 'ba_date{해당 날짜}2.csv' 파일에 저장되었습니다.")

In [ ]:
# 함수 실행
process_order_with_released_suit(df, order, count, shut,date)

### 기존 방법과 비교  
- 아래부터는 실행 필요 없음.  
- 작업량 균등 분배가 개선됨을 확인하는 용도.

In [ ]:
origin = pd.read_excel("data/BA_Sample_Samsung_0421_verificated_1.xlsx",usecols=['온라인주문번호의 순서','60슈트씩','수량'],sheet_name="출하_4월21일자")
origin

In [ ]:
order_origin = origin.groupby('온라인주문번호의 순서')['수량'].sum().reset_index()
order_origin

In [ ]:
workload_origin = []
for s in range(shut):
    workload_origin.append(order_origin[order_origin['온라인주문번호의 순서']%shut ==s]['수량'].sum())
print(workload_origin)

In [ ]:
import numpy as np
# 1번 방식
print(np.var(workload))
print(np.var(workload_origin))

In [ ]:
# 2번 방식
print(np.var(workload))

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
from matplotlib import rc

rc('font', family='AppleGothic')

workload_counter = Counter(workload)
workload_origin_counter = Counter(workload_origin)

values_workload = list(workload_counter.elements())
values_workload_origin = list(workload_origin_counter.elements())
plt.figure(figsize=(6, 4))

plt.hist(values_workload, bins=range(min(values_workload + values_workload_origin), max(values_workload + values_workload_origin) + 2), alpha=0.5, label='균등 할당')
plt.hist(values_workload_origin, bins=range(min(values_workload + values_workload_origin), max(values_workload + values_workload_origin) + 2), alpha=0.5, label='랜덤 할당')

plt.xlabel('슈트별 작업량')
plt.ylabel('빈도')
plt.title('기존 랜덤 할당 VS 균등 할당')
plt.legend(loc='upper right')

plt.show()